In [1]:
import json
import numpy as np
import pandas as pd
import requests

from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.models import HoverTool,LabelSet,ColumnDataSource
from bokeh.plotting import figure
from bokeh.server.server import Server
from bokeh.tile_providers import get_provider, STAMEN_TERRAIN


def wgs84_to_web_mercator_point(lon, lat):
    """ Function to convert GCS WGS84 to web marcator point. """
    
    k = 6378137  # WGS84 oblate spheroid with this equatorial radius in meters.
    x= lon * (k * np.pi / 180.0)
    y= np.log(np.tan((90 + lat) * np.pi / 360.0)) * k
    
    return x,y


def wgs84_to_web_mercator(df, lon='long', lat='lat'):
    """ Function to get Dataframe. """
    
    k = 6378137  # WGS84 oblate spheroid with this equatorial radius in meters.
    df['x'] = df[lon] * (k * np.pi / 180.0)
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360.0)) * k
    
    return df


# Madrid min and max longitude & latitude.
lon_min, lat_min = -3.6400, 40.3900
lon_max, lat_max = -3.7650, 40.4400

# COORDINATE CONVERSION
xy_min = wgs84_to_web_mercator_point(lon_min, lat_min)
xy_max = wgs84_to_web_mercator_point(lon_max, lat_max)

# COORDINATE RANGE IN WEB MERCATOR
x_range, y_range=([xy_min[0], xy_max[0]], [xy_min[1], xy_max[1]])


def citizens_tracking(doc):
    """ Init bokeh column data source. """
    
    citizen_source = ColumnDataSource(
        {
            'id': [],
            'name': [],
            'age': [],
            'time_position': [],
            'long': [],
            'lat': [],
            'x': [],
            'y':[],
            'url': []
        }
    )
    
    def update():
        """ Updating citizens data. """
        
        response = requests.get('http://localhost:9000/citizens').json()
        
        # with open('data/citizens_tracking.json') as data:
        #     response = json.load(data)
        
        # CONVERT TO PANDAS DATAFRAME
        col_name=['id', 'name', 'age', 'time_position', 'long', 'lat']
       
        citizen_df = pd.DataFrame(response['citizens']) 
        citizen_df = citizen_df.loc[:, 0:5] 
        citizen_df.columns = col_name
        wgs84_to_web_mercator(citizen_df)
        citizen_df = citizen_df.fillna('No Data')
        icon_url = 'http://icons.iconarchive.com/icons/custom-icon-design/flatastic-11/512/User-red-icon.png'
        citizen_df['url'] = icon_url
        
        # CONVERT TO BOKEH DATASOURCE AND STREAMING
        n_roll = len(citizen_df.index)
        citizen_source.stream(citizen_df.to_dict(orient='list'), n_roll)
        
    # CALLBACK UPATE IN AN INTERVAL
    doc.add_periodic_callback(update, 5000)  # 5000 ms/10000 ms for registered user.
    
    # PLOT CITIZEN POSITION
    p = figure(
        x_range=x_range,
        y_range=y_range,
        x_axis_type='mercator',
        y_axis_type='mercator',
        sizing_mode='scale_width',
        plot_height=300
    )
    tile_prov = get_provider(STAMEN_TERRAIN)
    
    p.add_tile(
        tile_prov,
        level='image'
    )
    p.image_url(
        url='url',
        x='x',
        y='y',
        source=citizen_source,
        anchor='center',
        angle_units='deg',
        h_units='screen',
        w_units='screen',
        w=40,
        h=40
    )
    p.circle(
        'x',
        'y',
        source=citizen_source,
        fill_color='red',
        hover_color='yellow',
        size=10,
        fill_alpha=0.8,
        line_width=0
    )

    # ADD HOVER TOOL AND LABEL
    my_hover=HoverTool()
    my_hover.tooltips=[
        ('DNI', '@id'),
        ('Name', '@name'),
        ('Age', '@age')
    ]
    labels = LabelSet(
        x='x',
        y='y',
        text='name',
        level='glyph',
        x_offset=5,
        y_offset=5,
        source=citizen_source,
        render_mode='canvas',
        background_fill_color='white',
        text_font_size='8pt'
    )
    p.add_tools(my_hover)
    p.add_layout(labels)
    doc.title='REAL TIME CITIZENS TRACKING'
    doc.add_root(p)
    
# SERVER CODE
apps = {'/': Application(FunctionHandler(citizens_tracking))}
server = Server(apps, port=8084)  # Define an unused port.
server.start()